In [1]:
import torch
import torch.nn as nn
import numpy as np
from data_process import get_data_loaders, getFeatures
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

## Training Model Example

In [2]:
# Data loader
input_len, output_len = 40, 20
all_features = getFeatures(covariates=False)
select_features = ['payload', 'wind_speed', 'wind_angle', 'linear_acceleration_x', 
                   'linear_acceleration_y', 'linear_acceleration_z', 'power']

data = pd.read_csv('flights.csv')
data, train_loader, val_loader, test_loader, d_split, scaler = get_data_loaders(data, input_len, output_len, test_size=0.2, val_size=0.2, 
                                                                                features=select_features, covariates=False)


['payload', 'wind_speed', 'wind_angle', 'linear_acceleration_x', 'linear_acceleration_y', 'linear_acceleration_z', 'power']


In [3]:
from model.LSTM_LSTM import LSTM_LSTM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

input_size = len(select_features)
output_size = len(select_features)
hidden_size = 32  
num_layers = 1

seq2seq = LSTM_LSTM(input_size, output_size, input_len, output_len, hidden_size, num_layers=num_layers).to(device)
saved_model_name = f'training/LSTM_LSTM_select_trained_model_{input_len}-{output_len}.pt'

cuda


In [4]:
# trainable parameters
pytorch_total_params = sum(p.numel() for p in seq2seq.parameters() if p.requires_grad)
print(pytorch_total_params)

10727


In [5]:
# train the model
# Loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(seq2seq.parameters(), lr=0.002)

# Training loop with validation and early stopping
num_epochs = 20
best_epoch = 0
best_val_loss = float('inf')
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    # Training phase
    seq2seq.train()
    total_train_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [TRAIN]")
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = seq2seq(inputs)  
        outputs = outputs.squeeze(-1) # (batch_size, output_len)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        progress_bar.set_postfix({'train_loss': loss.item()})

    average_train_loss = total_train_loss / len(train_loader)
    train_losses.append(average_train_loss)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Average Training Loss: {average_train_loss:.6f}")

    # Validation phase
    seq2seq.eval()
    total_val_loss = 0
    progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [VAL]")
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = seq2seq(inputs)  
        outputs = outputs.squeeze(-1) # (batch_size, output_len)

        loss = criterion(outputs, targets)

        total_val_loss += loss.item()
        progress_bar.set_postfix({'val_loss': loss.item()})
    
    average_val_loss = total_val_loss / len(val_loader)
    val_losses.append(average_val_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Validation Loss: {average_val_loss:.6f}")

    # Save the model with least validation loss
    if average_val_loss < best_val_loss:
        best_epoch = epoch + 1
        best_val_loss = average_val_loss
        torch.save(seq2seq.state_dict(), saved_model_name)

Epoch 1/20 [TRAIN]: 100%|██████████| 2239/2239 [00:34<00:00, 64.37it/s, train_loss=0.00835]


Epoch 1/20, Average Training Loss: 0.007666


Epoch 1/20 [VAL]: 100%|██████████| 750/750 [00:04<00:00, 160.46it/s, val_loss=0.00876]


Epoch 1/20, Average Validation Loss: 0.006360


Epoch 2/20 [TRAIN]: 100%|██████████| 2239/2239 [00:34<00:00, 65.10it/s, train_loss=0.00594]


...

Epoch 9/20 [TRAIN]: 100%|██████████| 2239/2239 [00:36<00:00, 61.11it/s, train_loss=0.00344]


Epoch 9/20, Average Training Loss: 0.003956


Epoch 9/20 [VAL]: 100%|██████████| 750/750 [00:04<00:00, 151.41it/s, val_loss=0.00339]


Epoch 9/20, Average Validation Loss: 0.003983


Epoch 10/20 [TRAIN]:  82%|████████▏ | 1834/2239 [00:31<00:08, 49.42it/s, train_loss=0.00625]

In [ ]:
# test the model
seq2seq.eval()
sum_mape = 0
sum_size = 0
for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    with torch.no_grad():
        outputs = seq2seq(inputs)  
        outputs = outputs.squeeze(-1)

    # print(r2_score(targets.cpu().T, outputs.cpu().T, multioutput='raw_values').shape)

    mape = torch.sum(torch.abs((outputs - targets) / targets)) * 100
    sum_mape += mape
    sum_size += targets.shape[0]*targets.shape[1]

print(f"{seq2seq.name} test MAPE: {sum_mape/sum_size}")

LSTM_LSTM test MAPE: 35.13011169433594
